In [83]:
import pandas as pd
from scraper import Scraper
import os

# 1 Scraping 

In [84]:
csv_name = '20_tokopedia_products.csv'

if csv_name in os.listdir():
    data = pd.read_csv(csv_name)
else:
    a = Scraper()
    data = a.get_data()
    a.driver.quit()
    
    data = pd.DataFrame(data)
    data.to_csv(csv_name, index=False)
pure_data = data.copy()

In [85]:
n = 4
print(data.Product[n], data.Description[n])

FLUX SET - Wedding Gift | Birthday Hampers | Kado | Hadiah FLUX SET :
- RTR : Classic Mini Air Diffuser / Humidifier 220ml Capacity (Include USB Cable for Air Humidifier & cotton sticks)
- RTR Aromatherapy Oil : 2 variants @10ml
- RTR : Bear Acrylic Night Lamp
- Corrugated white box 22x22cm
* FREE GREETING CARD


In [86]:
len(data)

20

In [87]:
data['id'] = data.index

# 2 Similarity

In [88]:
import contractions
import re
import nltk
nltk.download('stopwords')

import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import Normalizer

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [89]:
# Cleaning the texts
def txtprocess(txt):
    # Lower the texts
    txt = str(txt).lower()
    # Remove contractions
    txt = contractions.fix(txt)
    
    # Just pick the alphabet
    txt = re.sub(r'[^a-zA-Z]', ' ', txt)
    # Fix unnecessary space
    txt = re.sub(' +', ' ', txt)
    
    txt = ' '.join(txt.split())
    
    return txt

data.Product = data.Product.map(txtprocess)
data.Description = data.Description.map(txtprocess)

In [90]:
# Cleaning stopwords
stop_words = set(nltk.corpus.stopwords.words('indonesian'))
stop_words.add('gift')
stop_words.add('hampers')
stop_words.add('hadiah')
stop_words.add('kado')
stop_words.add('x')

def remove_stopwords(txt):
    no_stopword_txt = [w for w in txt.split() if not w in stop_words]
    return ' '.join(no_stopword_txt)

data.Product = data.Product.map(remove_stopwords)
data.Description = data.Description.map(remove_stopwords)

In [91]:
data['char_count'] = data['Description'].map(len)

In [92]:
data.Price = data.Price.str[2:].str.replace('.', '').astype(int)

In [93]:
# For product and description
tfidf_product = TfidfVectorizer()
product_vectors = tfidf_product.fit_transform(data.Product).toarray()

tfidf_description = TfidfVectorizer()
description_vectors = tfidf_description.fit_transform(data.Description).toarray()

product_similarity_matrix = cosine_similarity(product_vectors)
description_similarity_matrix = cosine_similarity(description_vectors)

# For prices and char count
normalized_prices = data.Price.values.reshape(1, -1)
normalized_char_count = data.char_count.values.reshape(1, -1)

scaler = Normalizer() 
normalized_prices = scaler.fit_transform(normalized_prices)
normalized_char_count = scaler.fit_transform(normalized_char_count)

normalized_prices = cosine_similarity(normalized_prices)
normalized_char_count = cosine_similarity(normalized_char_count)

# Weight
weight_product = 0.4
weight_description = 0.3
weight_prices = 0.2
weight_char_count = 0.1

# Combined Similarity
combined_similarity_matrix = (weight_product * product_similarity_matrix) + (weight_description * description_similarity_matrix) + (weight_prices * normalized_prices) + (weight_char_count * normalized_char_count)

j = 0
for i in range(len(combined_similarity_matrix)):
    top_3_similar_indices = combined_similarity_matrix[i].argsort()[::-1][1:4]
    top_3_similar_products = data.iloc[top_3_similar_indices]['Product']
#     if j!=19 and j!=3 and j!=4 and j!=1 and j!=12 and j!=13:
#         j+=1
#         continue
    print('Product ID', j)
    j+=1
    print(f"Top 3 similar products to '{data.iloc[i]['Product']}':")
    print(top_3_similar_products)
    print("\n")

Product ID 0
Top 3 similar products to 'souvenir pernikahan wedding':
14               souvenir pernikahan wedding
16    souvenir mug couple pernikahan wedding
19                 wedding pernikahan handuk
Name: Product, dtype: object


Product ID 1
Top 3 similar products to 'mug couple wedding':
16    souvenir mug couple pernikahan wedding
14               souvenir pernikahan wedding
0                souvenir pernikahan wedding
Name: Product, dtype: object


Product ID 2
Top 3 similar products to 'zwitsal baby set box paket perlengkapan bayi':
8                  newborn set baby bayi bayi
3              bayi lahiran bayi newborn baby
15    kelahiran bayi baby newborn set newborn
Name: Product, dtype: object


Product ID 3
Top 3 similar products to 'bayi lahiran bayi newborn baby':
8                            newborn set baby bayi bayi
15              kelahiran bayi baby newborn set newborn
6     bayi baby girl boy lahiran perempuan laki laki...
Name: Product, dtype: object


Product I

# 3 Product Description Enhancement

In [ ]:
import openai

In [44]:
key = "YOUR_API_KEY"

In [132]:
# prompt engineering trial and error
# message = 'perbaiki deskripsi produk berikut sehingga menarik bagi pembeli tanpa menambah kalimat \n\n'
# message = 'perbaiki deskripsi produk berikut sehingga menarik bagi pembeli dan mempunyai informasi yang padat \n\n'
message = 'perbaiki deskripsi produk berikut sehingga menarik bagi pembeli namun mempunyai informasi yang padat \n\n'
message = message+pure_data.Description[10]

In [133]:
print(message)

perbaiki deskripsi produk berikut sehingga menarik bagi pembeli namun mempunyai informasi yang padat 

Mangkuk Jepang Set dengan sumpit bambu / Japanese Bowl Set 
📢📢Perlu Diperhatikan!! 
- Semua Varian sudah termasuk gift box dan sumpit
- untuk set 2 mangkok, motif random diambil dr varian set 4  jenis produk yg dipilih ( tidak bisa pilih motif )
- untuk sakura mix set 2, warna random tidak bisa pilih
➡️ Gratis Kartu Ucapan dalam Bentuk Sticker print, mohon ucapan ditulis langsung di note / catatan produk.
jika tidak ditulis, maka akan dikirim tanpa sticker ucapan 
▶️▶️untuk mangkok set 6, link ada di produk sebelah
Semua mangkuk keramik adalah handmade dan hand painted sehingga titik kecil, titik hitam dll adalah hal yg normal terjadi ketika proses produksi.  Hal ini tidak akan mempengaruhi kualitas mangkok
mangkuk juga sudah didesign anti slip, bagian paling bawah sengaja tidak diglaze
Glaze Artistik Ceramic terbuat dari tanah liat langsung yang harus di tempa dan dipanaskan disuhu 1

In [134]:
openai.api_key = key

response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[{'role':'user', 'content':f'{message}'}],
  temperature=0,
  max_tokens=1024
)

In [135]:
print(response['choices'][0]['message']['content'])

Mangkuk Jepang Set dengan Sumpit Bambu / Japanese Bowl Set

🎁🎁 Perhatian! Setiap varian sudah termasuk kotak hadiah dan sumpit.
- Untuk set 2 mangkuk, motif akan dipilih secara acak dari varian set 4 jenis produk yang dipilih (tidak dapat memilih motif).
- Untuk Sakura Mix Set 2, warna akan dipilih secara acak dan tidak dapat dipilih.

➡️ Dapatkan Kartu Ucapan Gratis dalam Bentuk Stiker cetak, harap tuliskan ucapan langsung di catatan produk.
Jika tidak ditulis, maka akan dikirim tanpa stiker ucapan.

▶️▶️ Untuk mangkuk set 6, silakan lihat link di produk sebelah.

Semua mangkuk keramik ini dibuat secara handmade dan dicat tangan sehingga adanya titik kecil, titik hitam, dll adalah hal yang normal dalam proses produksi. Hal ini tidak akan mempengaruhi kualitas mangkuk.
Mangkuk ini juga didesain anti slip, dengan bagian bawah yang sengaja tidak dilapisi glaze.

Keramik Glaze Artistik terbuat dari tanah liat langsung yang harus ditempa dan dipanaskan pada suhu 1300 derajat, sehingga mang